In [ ]:
%matplotlib ipympl
from pathlib import Path

import matplotlib.pyplot as plt
import napari
import numpy as np
import tifffile
from aicsimageio.readers import TiffGlobReader
from ipywidgets import Output
from mda_simulator import ImageGenerator
from mda_simulator.mmcore import FakeDemoCamera
from mpl_interactions import hyperslicer
from napari_broadcastable_points import BroadcastablePoints
from pymmcore_plus import CMMCorePlus
from raman_mda_engine import RamanEngine, RamanTiffAndNumpyWriter, fakeAcquirer
from useq import MDASequence

v = napari.Viewer()
dw, main_window = v.window.add_plugin_dock_widget("napari-micromanager")

core = CMMCorePlus.instance()
core.loadSystemConfiguration("../tests/test-config.cfg")

gen = ImageGenerator(N=4000)


# Create an object that will modify the `snap` method of the CMMCorePlus
# instance to return images from our ImageGenerator
fake_cam = FakeDemoCamera(
    gen,
    timing=0.5,  # how many real world seconds to wait to step the ImageGenerator time
    core=core,
)


metadata = {
    "raman": {
        "z": "center",
        "channel": "BF",
    },
}
init_sequence = MDASequence(
    channels=["BF"],
    time_plan={"interval": 0.5, "loops": 2},
    z_plan={"range": 50, "step": 5},
    axis_order="tpcz",
    stage_positions=[(0, 1, 1), (512, 128, 0)],
)

# points = v.add_points(ndim=4)

In [ ]:
engine = RamanEngine(spectra_collector=fakeAcquirer())
core.register_mda_engine(engine)

In [ ]:
from raman_mda_engine.aiming import SimpleGridSource

In [ ]:
engine.aiming_sources.append(SimpleGridSource(15,15))

In [ ]:
engine.aiming_sources

In [ ]:
mda_output = Output()
with mda_output:
    core.run_mda(init_sequence).join()

In [ ]:
core.getImageWidth(), core.getImageHeight()

In [ ]:
cell_points = BroadcastablePoints(
    None,
    #               t, c, z
    broadcast_dims=(0, 2, 3),
    ndim=6,
    name="Cells",
    size=15,
    face_color="#aa0000ff",
    edge_color="#5500ffff",
)
bkd_points = BroadcastablePoints(
    None,
    #               t, c, z
    broadcast_dims=(0, 2, 3),
    ndim=6,
    name="Bkd points",
    size=15,
    face_color="#2d1f7f",
    edge_color="#5500ffff",
)

v.add_layer(cell_points)
v.add_layer(bkd_points)
# engine.points_layers = [cell_points, bkd_points]

In [ ]:
from raman_mda_engine.aiming import PointsLayerSource

In [ ]:
points_sources = PointsLayerSource(cell_points, name='cells'), PointsLayerSource(cell_points, name='bkd')

In [ ]:
engine.aiming_sources.extend(points_sources)

In [ ]:
engine.aiming_sources


In [ ]:
points_sources[0]._points

In [ ]:
points_sources[0]._points.last_displayed()


In [ ]:
points_sources[0].get_current_points()

In [ ]:
bkd_points.data[bkd_points._slice_data(v.dims.current_step[:-2])[0]][:,-2:]

In [ ]:
bkd_points.

In [ ]:
basic_sequence = MDASequence(
    metadata={"raman": {"z": "center"}},
    channels=["BF"],
    time_plan={"interval": 60, "loops": 3},
    # z_plan={"range": 30, "step": 10},
    z_plan={"relative": [-15, 0, 15]},
    axis_order="tpcz",
    stage_positions=[(0, 1, 1), (512, 128, 0)],
)
core.run_mda(basic_sequence)

In [ ]:
cell_points.loaded

In [ ]:
v.dims.current_step

In [ ]:
# bkd_points.data
bkd_points.data[bkd_points._slice_data(v.dims.current_step)[0]][:,-2:]

In [ ]:
bkd_points._view_data

In [ ]:
bkd_points.data.shape

In [ ]:
plt.figure()
plt.imshow(tifffile.imread("data/run_5/t000_p000_c000_z000.tiff"))